In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import os
import pickle

In [2]:
df_january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df_january.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
len(df_january.columns)

19

Q1 : Yellow Taxi Trip Records January 2023 has 19 columns

In [6]:
df_january['duration'] = df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime 

In [7]:
df_january.duration = df_january.duration.apply(lambda td : td.total_seconds()/60)

In [8]:
df_january.duration.std()

42.59435124195458

Q2 : Standard deviation of the duration in tellow taxi trip records of January 2023 is 42.59. 

In [9]:
df_january.duration.sample(5)

396839     13.633333
2704875    32.483333
2003458     8.900000
1815461    11.166667
270112     31.616667
Name: duration, dtype: float64

In [10]:
((df_january.duration >= 1) & (df_january.duration <=60)).mean()

0.9812202822125979

Q3 : 98% of the data has the duration between 1 minute and 60 minutes.

In [11]:
def read_dataframe(file_name):
    df = pd.read_parquet(file_name)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td : td.total_seconds()/60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [12]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Only PULocationID, DOLocationID are chosen as features

In [13]:
feature = ['PULocationID', 'DOLocationID']
target = ['duration']

In [14]:
df_train[feature+target].sample(5)

,PULocationID,DOLocationID,duration
2850539,137,186,7.516667
2834533,236,162,16.900000
1232280,262,239,11.766667
665383,236,161,10.333333
2888908,161,238,10.116667


In [15]:
df_val[feature+target].sample(5)

,PULocationID,DOLocationID,duration
430410,48,113,12.366667
2426401,148,148,5.383333
1629673,246,255,35.266667
2186659,151,151,3.966667
615341,87,249,13.033333


In [16]:
dv = DictVectorizer()

train_dict = df_train[feature].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[feature].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [17]:
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
X_train.shape[1]

515

Q4 : The feature matrix has 515 columns.

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [20]:
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.811818743246608

Q5 : The RMSE of the linear regression model is 7.81. 